In [46]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, models
from PIL import Image
import numpy as np
from collections import Counter

In [47]:
def create_model1():
    model = models.resnet50(pretrained=False)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 14)  # Assuming 10 classes
    return model
# Function to load a model and its state dict from a .pth file
def load_model1(model_path):
    model = create_model1()  # Create the model architecture
    state_dict = torch.load(model_path)  # Load the state dict
    model.load_state_dict(state_dict)  # Load the state dict into the model
    model.eval()  # Set the model to evaluation mode
    return model
def create_model2():
    model = models.resnet50(pretrained=False)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 47)  # Assuming 10 classes
    return model
def load_model2(model_path):
    model = create_model2()  # Create the model architecture
    state_dict = torch.load(model_path)  # Load the state dict
    model.load_state_dict(state_dict)  # Load the state dict into the model
    model.eval()  # Set the model to evaluation mode
    return model

def create_model3():
    model = models.resnet50(pretrained=False)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 6)  # Assuming 10 classes
    return model
def load_model3(model_path):
    model = create_model3()  # Create the model architecture
    state_dict = torch.load(model_path)  # Load the state dict
    model.load_state_dict(state_dict)  # Load the state dict into the model
    model.eval()  # Set the model to evaluation mode
    return model

# Function to load and preprocess the image
def preprocess_image(image_path):
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    img = Image.open(image_path).convert('RGB')
    img = preprocess(img)
    img = img.unsqueeze(0) # Add batch dimension
    return img

In [48]:
def combine_models():
    # Load the models
    model_paths = ['/newstorage5/aayesha/deep_learning/classification/style_classification/resnet50_custom_style_multiclass_model_20Epochs.pth', '/newstorage5/aayesha/deep_learning/classification/pattern_classification/resnet50_custom_pattern_multiclass_model_20Epochs.pth', '/newstorage5/aayesha/deep_learning/classification/fabric_classification/resnet50_custom_fabric_multiclass_model_20Epochs.pth']
    #models = [load_model(model_path) for model_path in model_paths]
    models = [load_model1(model_paths[0]), load_model2(model_paths[1]), load_model3(model_paths[2])]
    return models

In [49]:
# Load and preprocess the input image 1
image_path = '/newstorage5/aayesha/deep_learning/classification/shirt1.jpg'
input_image = preprocess_image(image_path)
input_image1 = input_image.to('cuda' if torch.cuda.is_available() else 'cpu')

In [50]:
# Load and preprocess the input image 2
image_path = '/newstorage5/aayesha/deep_learning/classification/shirt2.jpg'
input_image = preprocess_image(image_path)
input_image2 = input_image.to('cuda' if torch.cuda.is_available() else 'cpu')

In [51]:
# Load and preprocess the input image 3
image_path = '/newstorage5/aayesha/red_round.jpg'
input_image = preprocess_image(image_path)
input_image3 = input_image.to('cuda' if torch.cuda.is_available() else 'cpu')

In [52]:
# Load and preprocess the input image 4
image_path = '/newstorage5/aayesha/red_zigzag.jpg'
input_image = preprocess_image(image_path)
input_image4 = input_image.to('cuda' if torch.cuda.is_available() else 'cpu')

In [53]:
input_images = [input_image1, input_image2, input_image3, input_image4]

In [54]:
# Perform inference with each model and extract the feature vector
def ext_feature_vec(input_image):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    outputs = []
    all_classes = []
    models = combine_models()
    for model in models:
        model = model.to(device)
        with torch.no_grad():
            # Assuming the models output the logits, features are extracted before the final classification layer
            # Remove the final classification layer (e.g., if it's a fully connected layer in ResNet)
            features = model(input_image)
            outputs.append(features)
            _, predicted = torch.max(features, 1)
            x = predicted.item()
        all_classes.append(x)
    return outputs

In [55]:
# Function to combine model outputs into a single vector
def combine_outputs_as_vector(outputs, method='concatenate'):
    if method == 'concatenate':
        combined_vector = torch.cat(outputs, dim=1)  # Concatenate along the feature dimension
    elif method == 'average':
        combined_vector = torch.mean(torch.stack(outputs), dim=0)
    else:
        raise ValueError("Invalid combination method provided.")
    return combined_vector

In [56]:
# Get combined vector representation of all given images
def get_all_vectors():
    img_test = []
    for j in range(len(input_images)):
        outputs = ext_feature_vec(input_images[j])
        combined_vector = combine_outputs_as_vector(outputs, method='concatenate')
        img_test.append(combined_vector.cpu().numpy())
    return img_test


In [57]:
# Calculate similarity between any two given vectors
def cosine_similarity(vector1, vector2):
    vector1 = vector1.flatten()
    vector2 = vector2.flatten()
    cos_sim = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
    return cos_sim

In [59]:
# Get similarities among images and return them
def get_all_sim(input_images):
    img_test = get_all_vectors()
    all_sim = []
    for k in range(len(img_test)):
        each_sim = []
        tracker = []
        for m in range(len(img_test)):
            if not np.array_equal(img_test[k], img_test[m]):
                each_sim.append(cosine_similarity(img_test[k], img_test[m]))
                tracker.append(m)       
        maxi = max(each_sim)
        idex_val = each_sim.index(maxi)
        maxi_m = tracker[idex_val]
        maxi_k = k
        all_sim.append([maxi, maxi_m, maxi_k])
    return [all_sim, img_test]
all_sim = get_all_sim(input_images)[0]
img_test = get_all_sim(input_images)[1]


/tmp/ipykernel_3617725/2464067799.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path)  # Load the state dict
/tmp/ipykernel_3617725/24640

In [60]:
def get_most_similar_items():
    most_freq = []
    for i in all_sim:
        most_freq.append(i[1]) 
    return most_freq

In [61]:
# Some calculations used for the later tasks

def most_frequent_number(numbers):
    # Create a Counter object to count the frequency of each number
    counter = Counter(numbers)
    # Get the most common number and its frequency
    most_common_number, frequency = counter.most_common(1)[0]
    return [most_common_number, frequency]

def get_most_frequent_item_and_frequency():
    numbers = get_most_similar_items()
    most_frequent, count = most_frequent_number(numbers)
    return [most_frequent, count]

def find_indices(arr, target):
    indices = []
    for index, value in enumerate(arr):
        if value == target:
            indices.append(index)
    return indices

In [64]:
def get_index_of_other_matching_items():
    most_frequent = get_most_frequent_item_and_frequency()[0]
    arr = get_most_similar_items()
    target = most_frequent
    rem_indices = find_indices(arr, target)
    return rem_indices

In [65]:
def get_sim_scores(all_sim):
        indices = get_index_of_other_matching_items()
        simi_val = []
        for j in indices:               
                simi_val.append([j, all_sim[j][0]])
        return simi_val

In [66]:
x = get_sim_scores(all_sim)
def get_max_sim_score(x):
    simi_arr = []
    for each in x:
        simi_arr.append(each[1])
    max_score = max(simi_arr)
    return([max_score, x[simi_arr.index(max_score)][0]])


In [67]:
x = get_sim_scores(all_sim)
def get_max_score_item(x):
    most_frequent = get_most_frequent_item_and_frequency()[0]
    simi_arr = []
    for each in x:
        simi_arr.append(each[1])
    max_score = max(simi_arr)
    return([[max_score, most_frequent, x[simi_arr.index(max_score)][0]], [max_score, x[simi_arr.index(max_score)][0], most_frequent], most_frequent, x[simi_arr.index(max_score)][0]])


In [68]:
def get_removed_items():
    removed_sim = get_max_score_item(x)[0:2]
    return removed_sim

In [69]:
def get_remain_sim_items():
    top_sim = get_max_score_item(x)[0]
    sec_sim = get_max_score_item(x)[1]
    all_sim_rem= all_sim
    all_sim_rem.remove(top_sim)
    all_sim_rem.remove(sec_sim)
    return all_sim_rem

In [70]:
def top_and_second_items():
    top_item = get_max_score_item(x)[2]
    second_item = get_max_score_item(x)[3]
    return([top_item, second_item])

In [71]:
def get_third_item_sim_wrt_first_two(remain_sim):
    top_item = top_and_second_items()[0]
    second_item = top_and_second_items()[1]
    #img_test = get_all_vectors()
    remain_itm_id = []
    #all_sim_rem = get_remain_sim_items()
    for each in remain_sim:
        remain_itm_id.append(each[2])
    all_cos = []
    for x in remain_itm_id:
        cos1 = cosine_similarity(img_test[x], img_test[top_item])
        cos2 = cosine_similarity(img_test[x], img_test[second_item])
        all_cos.append([x, cos1 + cos2])
    return all_cos

In [72]:
def get_item_third_place(remain_sim):
   all_cos = get_third_item_sim_wrt_first_two(remain_sim)
   third_place_item = max(all_cos, key=lambda x : x[1])
   return third_place_item

In [73]:
def get_first_three_items(remain_sim):
    top_item = top_and_second_items()[0]
    second_item = top_and_second_items()[1]
    third_item = get_item_third_place(remain_sim)[0]
    return (top_item, second_item, third_item)

In [74]:
def entire_workflow_for_three_selections():
    models = combine_models()
    # Load and preprocess the input image
    image_path = '/newstorage5/aayesha/deep_learning/classification/shirt1.jpg'
    input_image = preprocess_image(image_path)
    input_image1 = input_image.to('cuda' if torch.cuda.is_available() else 'cpu')
    # Load and preprocess the input image
    image_path = '/newstorage5/aayesha/deep_learning/classification/shirt2.jpg'
    input_image = preprocess_image(image_path)
    input_image2 = input_image.to('cuda' if torch.cuda.is_available() else 'cpu')
    # Load and preprocess the input image
    image_path = '/newstorage5/aayesha/red_round.jpg'
    input_image = preprocess_image(image_path)
    input_image3 = input_image.to('cuda' if torch.cuda.is_available() else 'cpu')
    # Load and preprocess the input image
    image_path = '/newstorage5/aayesha/red_zigzag.jpg'
    input_image = preprocess_image(image_path)
    input_image4 = input_image.to('cuda' if torch.cuda.is_available() else 'cpu')
    input_images = [input_image1, input_image2, input_image3, input_image4]
    remain_sim = get_remain_sim_items()
    top_three_recs = get_first_three_items(remain_sim)
    return top_three_recs

In [75]:
top_three_recs = entire_workflow_for_three_selections()
print(top_three_recs)

/tmp/ipykernel_3617725/2464067799.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path)  # Load the state dict
/tmp/ipykernel_3617725/24640

(0, 3, 2)
